Fit QuantifierRecommender and save the resulting Meta-Table

In [1]:
from quantifier_recommender import QuantifierRecommender

recommender = QuantifierRecommender(supervised=True)
# recommender.fit(datasets_path="./datasets/", train_data_path="./data/train_data/", test_data_path="./data/test_data/")
# recommender.save_meta_table("./recommender_data/meta_table.h5")

Evaluate QuantifierRecommender with Leave-One-Out

In [1]:
from quantifier_recommender import QuantifierRecommender

recommender = QuantifierRecommender(supervised=True)
recommender.load_fit_meta_table("./recommender_data/meta_table.h5")
recommender_eval, quantifiers_eval = recommender.leave_one_out_evaluation("./plot_data/recommender_evaluation_table.csv",
                                                                          "./plot_data/quantifiers_evaluation_table.csv")

Evaluate the KQuantifier for each dataset using APP

In [2]:
from k_quantifier import KQuantifier

k_quantifier = KQuantifier()
k_quantifier_eval = k_quantifier.evaluation(recommender_eval, quantifiers_eval, "./plot_data/k_quantifier_evaluation_table.csv")

Join the Quantifiers Evaluation Table with the KQuantifier Evaluation Table

In [3]:
import pandas as pd

complete_evaluation_table = pd.concat([quantifiers_eval, k_quantifier_eval], axis=0)
complete_evaluation_table.to_csv("./plot_data/complete_evaluation_table.csv", index=False)

Example of using the KQuantifier

In [3]:
from k_quantifier import KQuantifier
import pandas as pd
import numpy as np
from utils_ import load_train_test_data
from sklearn.preprocessing import MinMaxScaler
import pdb

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

dataset = pd.read_csv("./datasets/AedesQuinx.csv")
scaler = MinMaxScaler()

y = dataset.pop(dataset.columns[-1])
y = y.to_numpy()
X = dataset.to_numpy()
# X = scaler.fit_transform(dataset)
# X = np.c_[X, y]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train2, y_train2, X_test2, y_test2 = load_train_test_data(dataset_name="AedesQuinx",
                                                            train_data_path="./data/train_data",
                                                            test_data_path="./data/test_data")


cancer = load_breast_cancer()

# Features (X) and labels (y)
X = cancer.data
y = cancer.target

# Split the dataset into training and test sets
# test_size=0.2 means 20% of data will go to the test set, 80% to the train set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

k_quantifier_median = KQuantifier(k=-1, method="median")
k_quantifier_weighted = KQuantifier(k=-1, method="weighted")

k_quantifier_median.fit(X_train2, y_train2)
k_quantifier_weighted.fit(X_train2, y_train2)

In [4]:
print(f"MEDIAN: {k_quantifier_median.predict(X_test2)}\nWEIGHTED: {k_quantifier_weighted.predict(X_test2)}\nTRUE: {sum(y_test2)/len(y_test2)}")

MEDIAN: 0.492949037167068
WEIGHTED: 0.49361363216148213
TRUE: 0.5008333333333334


In [13]:
import pandas as pd

k_eval = pd.read_csv("./plot_data/k_quantifier_evaluation_table.csv")
k_eval = k_eval[k_eval['dataset'] == 'AedesQuinx']
k_eval = k_eval[k_eval['alpha'] == 0.11]
k_eval

,quantifier,dataset,sample_size,sampling_seed,iteration,alpha,pred_prev,abs_error,run_time
20,Top-1,AedesQuinx,100,5393,1,0.11,0.06,0.05,0.221366
21,Top-1,AedesQuinx,100,8792,2,0.11,0.05,0.06,0.222180
22,Top-1,AedesQuinx,100,8433,3,0.11,0.14,0.03,0.221634
23,Top-1,AedesQuinx,100,7513,4,0.11,0.01,0.10,0.220693
24,Top-1,AedesQuinx,100,2612,5,0.11,0.04,0.07,0.221747
...,...,...,...,...,...,...,...,...,...
40025,Top-9,AedesQuinx,100,7041,6,0.11,0.11,0.00,1.668814
40026,Top-9,AedesQuinx,100,9555,7,0.11,0.06,0.05,1.669843
40027,Top-9,AedesQuinx,100,6235,8,0.11,0.07,0.04,1.667040
40028,Top-9,AedesQuinx,100,5486,9,0.11,0.09,0.02,1.691945
